# Feature Engineering Task 2

In [3]:
import os
import pandas as pd

path_dataframes = "../dataset/dataframes"
file_path = os.path.join(path_dataframes, "test_data.pkl")

# Carica il DataFrame usando pd.read_pickle
df_test = pd.read_pickle(file_path)

In [4]:
print("Contenuto del DataFrame:")
print(df_test.head())

print("\nDimensione del DataFrame:")
print(df_test.shape)

Contenuto del DataFrame:
    TIME   P1   P2   P3   P4   P5   P6   P7  Case
0  0.000  2.0  2.0  2.0  2.0  2.0  2.0  2.0   178
1  0.001  2.0  2.0  2.0  2.0  2.0  2.0  2.0   178
2  0.002  2.0  2.0  2.0  2.0  2.0  2.0  2.0   178
3  0.003  2.0  2.0  2.0  2.0  2.0  2.0  2.0   178
4  0.004  2.0  2.0  2.0  2.0  2.0  2.0  2.0   178

Dimensione del DataFrame:
(55246, 9)


Metriche nel dominio del tempo:

In [5]:
import numpy as np

def time_domain_metrics(signal):
    """
    Calcola le metriche nel dominio del tempo per un array 1D (signal).
    Restituisce un dizionario con i valori calcolati.
    """
    metrics = {}
    metrics["mean"] = np.mean(signal)
    metrics["median"] = np.median(signal)
    metrics["p25"] = np.percentile(signal, 25)
    metrics["p75"] = np.percentile(signal, 75)
    metrics["variance"] = np.var(signal)
    metrics["line_integral"] = np.trapezoid(signal)  # approssima l'integrale
    metrics["min"] = np.min(signal)
    metrics["max"] = np.max(signal)
    return metrics

Metriche nel dominio della frequenza (FFT)

In [7]:
def frequency_domain_metrics(signal, sampling_rate=1000):
    """
    Calcola alcune metriche spettrali (fft) per un array 1D (signal).
    Restituisce un dizionario con i valori calcolati.
    Puoi estendere questa funzione con SNR, SINAD, ecc.
    """
    fft_result = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal), d=1/sampling_rate)
    
    # Teniamo solo le frequenze positive (>0), escludendo la componente DC
    positive_indices = np.where(freqs > 0)
    fft_result = fft_result[positive_indices]
    freqs = freqs[positive_indices]
    
    power_spectrum = np.abs(fft_result) ** 2
    
    # Esempio di metriche spettrali basilari
    metrics = {}
    metrics["peak_value"] = np.max(power_spectrum)
    metrics["peak_freq"] = freqs[np.argmax(power_spectrum)]
    metrics["sum_power_spectrum"] = np.sum(power_spectrum)
    metrics["std_power_spectrum"] = np.std(power_spectrum)
    
    # (Facoltativo) Esempio di RMS nel dominio frequenziale
    metrics["rms_freq"] = np.sqrt(np.mean(power_spectrum))
    
    return metrics

Funzione per aggregare un singolo Case

In [8]:
def aggregate_case(case_group, sampling_rate=1000):
    """
    case_group: subset del DataFrame per un singolo Case
    Ritorna un dizionario con le metriche calcolate per le colonne che iniziano con "P".
    """
    result = {}
    
    # Consideriamo solo le colonne di segnale che iniziano con "P"
    signal_columns = [col for col in case_group.columns if col.startswith("P")]
    
    for col in signal_columns:
        signal = case_group[col].values
        
        # Calcolo delle metriche nel dominio del tempo
        td_metrics = time_domain_metrics(signal)
        # Calcolo delle metriche nel dominio della frequenza
        fd_metrics = frequency_domain_metrics(signal, sampling_rate=sampling_rate)
        
        # Inseriamo le metriche nel dizionario finale con un prefisso per ciascun segnale
        for k, v in td_metrics.items():
            result[f"{col}_time_{k}"] = v
        for k, v in fd_metrics.items():
            result[f"{col}_freq_{k}"] = v
    
    return result

Applica l’aggregazione a tutto il DataFrame

In [9]:
import pandas as pd
import os

# Raggruppa per 'Case'
grouped = df_test.groupby("Case")

# Lista dove salveremo i risultati (dizionari)
aggregated_rows = []

for case, group in grouped:
    row_dict = aggregate_case(group, sampling_rate=1000)
    # Aggiungi la colonna 'Case' in modo esplicito
    row_dict["Case"] = case
    aggregated_rows.append(row_dict)

# Costruisci il DataFrame finale
df_aggregated = pd.DataFrame(aggregated_rows)

Risultato finale

In [10]:
print("DataFrame aggregato:")
print(df_aggregated.head())

print("\nDimensioni finali:", df_aggregated.shape)

DataFrame aggregato:
   P1_time_mean  P1_time_median  P1_time_p25  P1_time_p75  P1_time_variance  \
0      1.984422        1.965596     1.898145     2.057667          0.090330   
1      1.984932        1.959335     1.893304     2.062764          0.093511   
2      1.984972        1.959743     1.893367     2.058572          0.095046   
3      1.985002        1.956308     1.893223     2.058859          0.095035   
4      1.984954        1.959155     1.893371     2.058777          0.095023   

   P1_time_line_integral  P1_time_min  P1_time_max  P1_freq_peak_value  \
0            2381.280903     0.561754     4.411926        10735.216150   
1            2381.879996     0.453799     4.103312        10976.103217   
2            2381.927644     0.436712     4.262760        11079.034425   
3            2381.962902     0.442751     4.279393        11076.215410   
4            2381.905815     0.437060     4.270896        11079.884937   

   P1_freq_peak_freq  ...  P7_time_variance  P7_time_line_i

In [11]:
print("\nLe colonne sono:", df_aggregated.columns)


Le colonne sono: Index(['P1_time_mean', 'P1_time_median', 'P1_time_p25', 'P1_time_p75',
       'P1_time_variance', 'P1_time_line_integral', 'P1_time_min',
       'P1_time_max', 'P1_freq_peak_value', 'P1_freq_peak_freq',
       'P1_freq_sum_power_spectrum', 'P1_freq_std_power_spectrum',
       'P1_freq_rms_freq', 'P2_time_mean', 'P2_time_median', 'P2_time_p25',
       'P2_time_p75', 'P2_time_variance', 'P2_time_line_integral',
       'P2_time_min', 'P2_time_max', 'P2_freq_peak_value', 'P2_freq_peak_freq',
       'P2_freq_sum_power_spectrum', 'P2_freq_std_power_spectrum',
       'P2_freq_rms_freq', 'P3_time_mean', 'P3_time_median', 'P3_time_p25',
       'P3_time_p75', 'P3_time_variance', 'P3_time_line_integral',
       'P3_time_min', 'P3_time_max', 'P3_freq_peak_value', 'P3_freq_peak_freq',
       'P3_freq_sum_power_spectrum', 'P3_freq_std_power_spectrum',
       'P3_freq_rms_freq', 'P4_time_mean', 'P4_time_median', 'P4_time_p25',
       'P4_time_p75', 'P4_time_variance', 'P4_time_line_

Esportazione in .csv e .pkl

In [13]:
df_aggregated.to_csv("../dataset/dataframes/test_data_aggregated.csv", index=False)
df_aggregated.to_pickle("../dataset/dataframes/test_data_aggregated.pkl")